# Sentiment Analysis

In [6]:
import pandas as pd
import numpy as np

In [7]:
df=pd.read_excel('train.xlsx')

# TFIDF

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

In [9]:
df.head()

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg


In [43]:
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git --upgrade --force-reinstall

  Cloning https://github.com/laxmimerit/preprocess_kgptalkie.git to /tmp/pip-req-build-fhn4yqro
  Running command git clone --filter=blob:none --quiet https://github.com/laxmimerit/preprocess_kgptalkie.git /tmp/pip-req-build-fhn4yqro
  Resolved https://github.com/laxmimerit/preprocess_kgptalkie.git to commit 0f56a6502679f6750b7f0ffed98f590d1f0246bd
  Preparing metadata (setup.py) ... done
  Created wheel for preprocess_kgptalkie: filename=preprocess_kgptalkie-0.11-py3-none-any.whl size=8174 sha256=384ff902926a7d0a9700c8444bfe99bb490383cf13616c74c8a5acb5578da0ee
  Stored in directory: /tmp/pip-ephem-wheel-cache-zxynamje/wheels/74/fe/05/0d013c54ae5e4afb77c6c480378063827c84f57ea5f554e072
Successfully built preprocess_kgptalkie
  Attempting uninstall: preprocess_kgptalkie
    Found existing installation: preprocess_kgptalkie 0.11
    Uninstalling preprocess_kgptalkie-0.11:
      Successfully uninstalled preprocess_kgptalkie-0.11


In [1]:
!pip install googletrans==4.0.0-rc1

In [10]:
import preprocess_kgptalkie as ps
import re
import string

def get_clean(x):
    x = str(x).lower()
    x = re.sub(r'\\', '', x)  # remove backslashes
    x = re.sub(r'_', ' ', x)  # replace underscores with space
    x = re.sub(r'\S+@\S+', '', x)  # remove emails
    x = re.sub(r'http\S+|www.\S+', '', x)  # remove URLs
    x = re.sub(r'<.*?>', '', x)  # remove HTML tags
    x = re.sub(r'[^a-zA-Z\s]', '', x)  # remove special characters and numbers
    x = re.sub(r'\s+', ' ', x).strip()  # remove extra spaces
    return x

In [11]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def get_clean_enhanced(x):
    x = get_clean(x)
    tokens = x.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return " ".join(tokens)

df['Reviews'] = df['Reviews'].apply(get_clean_enhanced)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
df['Reviews'] = df['Reviews'].apply(get_clean)

In [13]:
df.head()

,Reviews,Sentiment
0,first tuned morning news thought wow finally e...,neg
1,mere thought going overboard aka babe ahoy mak...,neg
2,movie fall well standard ultimately answer lie...,neg
3,wow thought steven segal movie bad every time ...,neg
4,story seen matter figure make proper storyboar...,neg


In [14]:
tfidf=TfidfVectorizer(max_features=5000)
x=df['Reviews']
y=df['Sentiment']

x=tfidf.fit_transform(x)

In [15]:
x

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1965651 stored elements and shape (25000, 5000)>

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=0)

In [17]:
clf = LinearSVC()
clf.fit(x_train, y_train)

LinearSVC()

In [18]:
y_pred = clf.predict(x_test)

In [19]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         neg       0.87      0.86      0.87      2480
         pos       0.87      0.87      0.87      2520

    accuracy                           0.87      5000
   macro avg       0.87      0.87      0.87      5000
weighted avg       0.87      0.87      0.87      5000



In [20]:
X = 'this movie is really great. Thanks a lot for making this movie'
x = get_clean(X)  # Fix: use 'X' not 'x'
vec = tfidf.transform([x])
print(clf.predict(vec))

['pos']


In [21]:
vec.shape

(1, 5000)

In [22]:
clf.predict(vec)

array(['pos'], dtype=object)

In [23]:
import pickle

In [24]:
pickle.dump(clf, open('model', 'wb'))